<a href="https://colab.research.google.com/github/jaybeberry9591/graph_theory_assignments/blob/main/kmeans_clustering_crime_2010_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import numpy as np

import pandas as pd

In [17]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/crime.csv")
df.head(10)

,year,unit,dacocity,robber,murder,speedy_trail,riot,women_children_repression,kidnapping,police_assault,burglary,theft,other_cases,arms_attack,explosive_attack,narcotic_act,smuggling,total_arms_explosive_narcotic_attack,total_cases
0,2010,DMP,47,220,245,363,3,1370,139,155,555,1915,7228,518,82,10535,144,11279,23519
1,2010,CMP,16,108,94,31,7,455,37,31,123,314,1831,51,0,866,99,1016,4063
2,2010,KMP,3,9,29,25,0,153,11,4,65,91,551,19,2,792,13,826,1767
3,2010,RMP,4,20,21,9,15,157,9,12,53,106,578,3,4,332,248,587,1571
4,2010,BMP,8,12,19,21,0,112,6,8,24,83,557,17,0,155,117,289,1139
5,2010,SMP,12,33,33,34,1,104,14,12,33,154,866,14,0,154,20,188,1484
6,2010,Dhaka Range,162,199,1153,362,7,4272,171,71,643,1477,19966,309,30,4459,993,5791,34274
7,2010,Mymensingh Range,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2010,Chittagong Range,153,122,639,245,32,2915,111,87,429,998,12985,235,20,4730,612,5597,24313
9,2010,Sylhet Range,85,43,245,75,17,848,41,19,186,524,5266,30,4,905,172,1111,8460


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   year                                  180 non-null    int64 
 1   unit                                  180 non-null    object
 2   dacocity                              180 non-null    int64 
 3   robber                                180 non-null    int64 
 4   murder                                180 non-null    int64 
 5   speedy_trail                          180 non-null    int64 
 6   riot                                  180 non-null    int64 
 7   women_children_repression             180 non-null    int64 
 8   kidnapping                            180 non-null    int64 
 9   police_assault                        180 non-null    int64 
 10  burglary                              180 non-null    int64 
 11  theft                           

In [19]:
import plotly.express as px
fig = px.scatter_matrix(df.drop(["year","unit"],axis=1),
width=1200, height=1600)
fig.show()

In [20]:
fig1 = px.scatter(df, x="murder", y="robber", color="total_arms_explosive_narcotic_attack",
                 size="total_cases")
fig1.update_layout(title="4 Features Representation")
fig1.show()

In [21]:
fig2 = px.scatter_3d(df, x="dacocity", y="robber",z="murder",
                     color="speedy_trail",size="riot")
fig2.update_layout(title="5 Features Representation")
fig2.show()

In [22]:
X=df.drop("unit",axis=1)
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)
inertia = []
for i in range(1,11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1,11),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,11],title="Cluster Number"),
                  yaxis={'title':'Inertia'},
                 annotations=[
        dict(
            x=3,
            y=inertia[2],
            xref="x",
            yref="y",
            text="Elbow!",
            showarrow=True,
            arrowhead=7,
            ax=20,
            ay=-40
        )
    ])

In [24]:
kmeans = KMeans(
        n_clusters=3, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
kmeans.fit(X)
clusters=pd.DataFrame(X,columns=df.drop("unit",axis=1).columns)
clusters['label']=kmeans.labels_
polar=clusters.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
fig4 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
fig4.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [25]:
pie=clusters.groupby('label').size().reset_index()
pie.columns=['label','value']
px.pie(pie,values='value',names='label',color=['blue','red','green'])